In [ ]:
%pip install numpy
%pip install pandas
%pip install tensorflow
%pip install numpy
%pip install scikit-learn

In [172]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam

In [157]:
batting_df = pd.read_csv('Batting.csv')
as_df = pd.read_csv('AllstarFull.csv')

In [158]:
batting_df = batting_df[batting_df['yearID'] >=1973]
batting_df = batting_df[batting_df['yearID'] <= 2014]
batting_df = batting_df[batting_df['AB'] >= 200]
batting_df.drop(['GIDP', 'SF' ,'SH', 'HBP', 'lgID' ,'teamID', 'stint' ,'CS'], inplace=True, axis=1)
batting_df.dropna(inplace=True)
batting_df.reset_index(inplace=True)
batting_df.drop(['index'], inplace=True, axis=1)
batting_df.head()

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,IBB
0,aaronha01,1973,120,392.0,84.0,118.0,12.0,1.0,40.0,96.0,1.0,68.0,51.0,13.0
1,ageeto01,1973,83,204.0,30.0,48.0,5.0,2.0,8.0,15.0,2.0,16.0,55.0,1.0
2,allendi01,1973,72,250.0,39.0,79.0,20.0,3.0,16.0,41.0,7.0,33.0,51.0,3.0
3,alomasa01,1973,136,470.0,45.0,112.0,7.0,1.0,0.0,28.0,25.0,34.0,44.0,1.0
4,aloufe01,1973,93,280.0,25.0,66.0,12.0,0.0,4.0,27.0,0.0,9.0,25.0,5.0


In [159]:
as_df = as_df[as_df['yearID'] >= 1974]
as_df = as_df[as_df['startingPos'] >1]
as_df.drop(['gameNum', 'gameID', 'teamID', 'lgID' ,'GP', 'startingPos'], inplace=True, axis=1)
as_df.reset_index(inplace=True)
as_df.drop(['index'], inplace=True, axis=1)
as_df.head()

,playerID,yearID
0,munsoth01,1974
1,allendi01,1974
2,carewro01,1974
3,robinbr01,1974
4,campabe01,1974


In [151]:
df = batting_df
df['ASNextYear'] = np.nan

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,IBB,ASNextYear
0,aaronha01,1973,120,392.0,84.0,118.0,12.0,1.0,40.0,96.0,1.0,68.0,51.0,13.0,NaN
1,ageeto01,1973,83,204.0,30.0,48.0,5.0,2.0,8.0,15.0,2.0,16.0,55.0,1.0,NaN
2,allendi01,1973,72,250.0,39.0,79.0,20.0,3.0,16.0,41.0,7.0,33.0,51.0,3.0,NaN
3,alomasa01,1973,136,470.0,45.0,112.0,7.0,1.0,0.0,28.0,25.0,34.0,44.0,1.0,NaN
4,aloufe01,1973,93,280.0,25.0,66.0,12.0,0.0,4.0,27.0,0.0,9.0,25.0,5.0,NaN


In [ ]:
for bIndex,bRow in batting_df.iterrows():
    for aIndex,aRow, in as_df.iterrows():
        if bRow['yearID'] == aRow['yearID'] -1 and bRow['playerID'] == aRow['playerID']:
            print(bRow['playerID'], bRow['yearID'])
            df.at[bIndex, 'ASNextYear']= 1
            as_df.drop(aIndex, inplace=True)
            
df = df.fillna(0)

In [153]:
df.to_csv('CapstoneData.csv', sep=',', index=False, encoding='utf-8')

# Nueral Network

In [169]:
df.head()

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,IBB,ASNextYear
0,aaronha01,1973,120,392.0,84.0,118.0,12.0,1.0,40.0,96.0,1.0,68.0,51.0,13.0,1.0
1,ageeto01,1973,83,204.0,30.0,48.0,5.0,2.0,8.0,15.0,2.0,16.0,55.0,1.0,0.0
2,allendi01,1973,72,250.0,39.0,79.0,20.0,3.0,16.0,41.0,7.0,33.0,51.0,3.0,1.0
3,alomasa01,1973,136,470.0,45.0,112.0,7.0,1.0,0.0,28.0,25.0,34.0,44.0,1.0,0.0
4,aloufe01,1973,93,280.0,25.0,66.0,12.0,0.0,4.0,27.0,0.0,9.0,25.0,5.0,0.0


In [196]:
features = df[['G','AB','R','H','2B','3B','HR','RBI','SB','BB','SO','IBB']]
label = df[['ASNextYear']]

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.5, random_state = 42)

std_scaler = StandardScaler()
X_train_std = std_scaler.fit_transform(X_train)
X_test_std = std_scaler.transform(X_test)

norm_scaler = MinMaxScaler()
X_train_norm = norm_scaler.fit_transform(X_train)
X_test_norm = norm_scaler.transform(X_test)

In [197]:
model = Sequential()
model.add(Input(shape=(12,)))
model.add(Dense(2048, 'relu'))
model.add(Dense(1024, 'relu'))
model.add(Dense(512,'relu'))
model.add(Dense(256, 'relu'))
model.add(Dense(128, 'relu'))
model.add(Dense(1, 'sigmoid'))

model_std = Sequential()
model_std.add(Input(shape=(12,)))
model_std.add(Dense(2048, 'relu'))
model_std.add(Dense(1024, 'relu'))
model_std.add(Dense(512,'relu'))
model_std.add(Dense(256, 'relu'))
model_std.add(Dense(128, 'relu'))
model_std.add(Dense(1, 'sigmoid'))

model_norm = Sequential()
model_norm.add(Input(shape=(12,)))
model_norm.add(Dense(2048, 'relu'))
model_norm.add(Dense(1024, 'relu'))
model_norm.add(Dense(512,'relu'))
model_norm.add(Dense(256, 'relu'))
model_norm.add(Dense(128, 'relu'))
model_norm.add(Dense(1, 'sigmoid'))

model.compile(optimizer=Adam(0.001), metrics=['accuracy'], loss = 'binary_crossentropy')
model_std.compile(optimizer=Adam(0.001), metrics=['accuracy'], loss = 'binary_crossentropy')
model_norm.compile(optimizer=Adam(0.001), metrics=['accuracy'], loss = 'binary_crossentropy')

In [198]:
model.fit(X_train, y_train, epochs=3, batch_size = 20)
#model_std.fit(X_train_std, y_train, epochs=5, batch_size=20)
#model_norm.fit(X_train_norm, y_train, epochs=5, batch_size=10)

Epoch 1/3
310/310 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.8951 - loss: 1.8348
Epoch 2/3
310/310 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9448 - loss: 0.1984
Epoch 3/3
310/310 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9393 - loss: 0.1999


In [199]:
print("UnScaled:", model.evaluate(X_train, y_train), model.evaluate(X_test,y_test))
print("STd:", model.evaluate(X_train_std, y_train), model.evaluate(X_test_std,y_test))
print("norm:", model.evaluate(X_train_norm, y_train), model.evaluate(X_test_norm,y_test))

194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9416 - loss: 0.1972
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9389 - loss: 0.2029
UnScaled: [0.18844446539878845, 0.9459983706474304] [0.1928071528673172, 0.9447049498558044]
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9416 - loss: 0.3889
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9389 - loss: 0.3902
STd: [0.38522735238075256, 0.9459983706474304] [0.38625943660736084, 0.9447049498558044]
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9416 - loss: 0.4303
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9389 - loss: 0.4321
norm: [0.4269346594810486, 0.9459983706474304] [0.4278915226459503, 0.9447049498558044]


In [205]:
jackiemvp = np.array([156,593,122,203,38,12,16,124,37,86,27,12])
jose23 = np.array([90,360,76,112,21,2,17,51,14,44,71,0])
jose22 = np.array([141,527,103,158,39,0,28,57,18,66,7,2])
jose21 = np.array([146,601,117,167,32,1,31,83,5,66,91,3])
model.predict(np.expand_dims(jackiemvp, axis=0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


array([[0.39900348]], dtype=float32)